# RESTful APIs - Django REST Framework 1

### Intro

The Django REST framework (http://www.django-rest-framework.org/) is a powerful and flexible toolkit for building Web APIs. Some reasons you might want to use a REST framework are:

- The Web browsable API is a huge usability win for your developers.
- Authentication policies including packages for OAuth1 and OAuth2.
- Serialization that supports both ORM and non-ORM data sources.
- Customizable all the way down - just use regular function-based views if you don't need the more powerful features.
- Extensive documentation, and great community support.
- Used and trusted by large companies such as Google, LinkedIn, Facebook, Twitter and Mozilla.

<img src="./images/rest.png" width="200" height="200" />


### Prerequisites

Before we do anything, we'll create a new virtual environment, using virtualenv. virtualenv is a tool to create isolated Python environments. The basic problem being addressed is one of dependencies and versions, and indirectly permissions. Imagine you have an application that needs version 1 of LibFoo, but another application requires version 2. How can you use both these applications? If you install everything into /usr/lib/python2.7/site-packages (or whatever your platform’s standard location is), it’s easy to end up in a situation where you unintentionally upgrade an application that shouldn’t be upgraded.

virtualenv creates an environment that has its own installation directories, that doesn’t share libraries with other virtualenv environments (and optionally doesn’t access the globally installed libraries either). This will make sure our package configuration is kept nicely isolated from any other projects we're working on.

Hence, choose a directory in your computer where you want to keep your virtual environments and type:

I am using '`env`' as the name of the environment for this particular practical but you can use any name you want. 

If you don't have virtualenv installed in your computer, install it:

Next you need to activate the virtual environment you just created, in Windows type:

in Linux flavor operating systems type the following to activate the virtual environment you just created:

Now that we're inside a virtualenv environment, we can install our package requirements.

### Creating a Django project and a model to work with

To get started, let's create a new Django project to work with. In a folder of your choosing type:

Now we can create an app that we'll use to create a simple Web API. We are going to create a RESTful API for an imaginary company that sells books and needs an API the has the functionality to displays, edit and delete its inventory data of books and associated metadata. So we start  by creating the corresponding app inside our Django project.

We'll need to add our new books app and the `rest_framework` app to `INSTALLED_APPS` inside the `resttutorial/settings.py` file:

In [1]:
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'rest_framework',
    'books.apps.BooksConfig',
]

For the purposes of this tutorial we're going to start by creating a simple `Book` model that is used to store book instances. Go ahead and edit the `resttutorial/books/models.py` file. 

In [ ]:
from django.db import models


class Book(models.Model):
    created = models.DateTimeField(auto_now_add=True)
    title = models.CharField(max_length=200, blank=True, default='')
    author = models.CharField(max_length=100)
    intro = models.TextField()
    price = models.DecimalField(decimal_places=2,max_digits=6)
    url = models.URLField()

    class Meta:
        ordering = ('created',)

Next, let's create an initial migration for our book model, and sync the database for the first time.

### Serializers

In computer science, in the context of data storage, serialization is the process of translating data structures or object state from a given programming language into a format that can be stored (for example, in a file or memory buffer, or transmitted across a network connection link) and reconstructed later in the same or another computer environment.

The first thing we need to get started on our Web API is to provide a way of serializing and deserializing the book instances into serialized representations such as json. We can do this by declaring serializers that work very similar to Django's forms. Create a file in the books directory named `resttutorial/books/serializers.py` and add the following:

In [ ]:
from rest_framework import serializers
from books.models import Book

class BookSerializer(serializers.ModelSerializer):
    class Meta:
        model = Book
        fields = ('id', 'title', 'author', 'intro', 'price', 'url')

Django REST framework makes it very easy to serialize a data model. You simply create a class that inherits from `serializers.ModelSerializer` and then specify in the `fields` variable the attributes from your model object that you want to include into the serialized representation of your model object.

### API views

REST framework provides two wrappers you can use to write API views.

- The @api_view decorator for working with function based views.
- The APIView class for working with class based views.

These wrappers provide a few bits of functionality such as making sure you receive Request instances in your view, and adding context to Response objects so that content negotiation can be performed. The wrappers also provide behaviour such as returning `405 Method Not Allowed` responses when appropriate, and handling any `ParseError` exception that occurs when accessing `request.data` with malformed input.

Writting our API views using class based views, rather than function based views is easier. This is a powerful pattern that allows us to reuse common functionality, and helps us keep our code DRY.

Translate the following views to your `resttutorial/books/views.py` file:

In [ ]:
from books.models import Book
from books.serializers import BookSerializer
from django.http import Http404
from rest_framework.views import APIView
from rest_framework.response import Response
from rest_framework import status


class BookList(APIView):
    """
    List all books with get method, or create a new book with post method.
    """
    def get(self, request, format=None):
        books = Book.objects.all()
        serializer = BookSerializer(books, many=True)
        return Response(serializer.data)

    def post(self, request, format=None):
        serializer = BookSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

class BookDetail(APIView):
    """
    Retrieve, update or delete a single book instance.
    """
    def get_object(self, pk):
        try:
            return Book.objects.get(pk=pk)
        except Book.DoesNotExist:
            raise Http404

    def get(self, request, pk, format=None):
        book = self.get_object(pk)
        serializer = BookSerializer(book)
        return Response(serializer.data)

    def put(self, request, pk, format=None):
        book = self.get_object(pk)
        serializer = BookSerializer(book, data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

    def delete(self, request, pk, format=None):
        book = self.get_object(pk)
        book.delete()
        return Response(status=status.HTTP_204_NO_CONTENT)

Finally we need to wire these views up. Create the `resttutorial/books/urls.py` file with the following content:

In [ ]:
from django.conf.urls import url
from rest_framework.urlpatterns import format_suffix_patterns
from books import views

urlpatterns = [
    url(r'^books/$', views.BookList.as_view()),
    url(r'^books/(?P<pk>[0-9]+)/$', views.BookDetail.as_view()),
]

urlpatterns = format_suffix_patterns(urlpatterns)

The last line adds support for format suffixes to our API endpoints. Using format suffixes gives us URLs that explicitly refer to a given format that we want the API to used to return the data (XML, JSON, HTML, etc). It effectively means our API will be able to handle URLs such as http://example.com/api/items/4/.json, http://example.com/api/items/4/.xml or http://example.com/api/items/4/.api.

We also need to wire up the root urlconf, in the `resttutorial/urls.py` file, to include our book app's URLs.

In [ ]:
from django.conf.urls import url, include
from django.contrib import admin

urlpatterns = [
    url(r'^admin/', admin.site.urls),
    url(r'^', include('books.urls')),
]

### Testing our Web API

We can test our API using httpie. Httpie is a user friendly http client that's written in Python. You can install httpie using pip:

Start your local server:

In another terminal window, activate the virtualenv, named `env` that we created at the beginning of this tutorial:

Now we are ready to test our Web API.

### Posting some data to our Web API

Let's fill our web API with some data. we will use `httpie` to generate HTTP POST requests to populate our API with data with the specific book instance data you want to update:

`httpie` should provide you with the HTTP response from the server. You should be getting a `201` status code indicating that the resource has been created on the server.

![](./images/httpResponse.png)

With the the previous `POST` request you generated an instance of a book in your Web API. Let's create another one.

And yet another one:

### Retrieving all the books or a single instance of a book from our  Web API

We can now get a list of all of the books in our Web application using a simple HTTP GET request:

We can also get information about a single book by its id:

We can also delete specific instances of a book in our database:

Or update the information for a specific instance (Note that I'm using nonsensical data for readability purposes):

You can also navigate your web API using your browser, by visiting:  http://127.0.0.1:8000/books/ or http://127.0.0.1:8000/books/1/

![](images/webapi.png)

Congratulations if you made it this far! you have achieved a lot in a single session, namely creating your first RESTful Web API. Web APIs are used by all the big online players (Google, Amazon, Facebook, Twitter, etc) to deliver functionality across the web. Now you can add to your CV that you have the technical skills to create and consume RESTful APIs.